In [24]:
import pandas as pd
import numpy as np

from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from itertools import product

import matplotlib.pyplot as plt

### Data

In [20]:
# r'Users/amandaquay/Box\ Sync/90904_FinalProject/SanJoaquinSalinityML/
pathway = 'data_average_after.csv'
df = pd.read_csv(pathway, index_col=0).dropna()
df = df.sort_values(by=['Field_ID']).sort_values(by=['Field_ID'])
# extract independent variables of use
df_small = df[['salinity', 'Field_ID', 'max_CRSI', 'elevation', 'aspect', 
               'slope', 'margins', 'average_temperature', 'total_precipitation']]
df_small.head()

,salinity,Field_ID,max_CRSI,elevation,aspect,slope,margins,average_temperature,total_precipitation
0,0.010,1.0,0.774290,66,328,1,1.0,285.08047,119.0
2,3.827,1.0,0.781937,66,152,0,1.0,285.08047,119.0
3,3.080,1.0,0.775323,66,151,0,1.0,285.08047,119.0
4,1.092,1.0,0.766537,66,149,0,1.0,285.08047,119.0
5,0.146,1.0,0.748702,66,148,0,1.0,285.08047,119.0


### Functions
###### These functions were mainly pulled from Paul Welle's work

In [13]:
def normalize_X(X_train, X_test):
    x_max = np.max(X_train, axis=0, keepdims=True)
    x_min = np.min(X_train, axis=0, keepdims=True)
    range_x = x_max-x_min
    range_x[range_x==0] = 1
    x_min[range_x==0] = 0
    X_train = (X_train - x_min) / range_x
    X_test = (X_test - x_min) / range_x
    return X_train, X_test

In [14]:
def plot_actual_vs_predicted(actual, predicted, ylim=(0,40), xlim=(0,40)):
    plt.scatter(actual, predicted, s=5)
    plt.xlim(xlim)
    plt.ylim(ylim)
    plt.plot((0,40),(0,40), linewidth=2, color='black')
    plt.xlabel('Actual Salinity (dS/m)')
    plt.ylabel('Predicted Salinity (dS/m)')
    plt.show()

In [15]:
def train_test_split_lofo(df, drop_columns, test_field=None):
    fields = df['Field_ID'].unique()
    if test_field is None: test_field = np.array([np.random.choice(fields)])
    else: test_field=np.array([test_field])
    train_fields = np.setdiff1d(fields, test_field)

    test_field_indices = np.where(df['Field_ID'].isin(test_field))[0]
    train_field_indices = np.where(df['Field_ID'].isin(train_fields))[0]

    y = df['salinity'].values.reshape(-1,1)
    X = df.drop(df.columns[drop_columns], axis=1).values

    y_train = y[train_field_indices].ravel()
    y_test = y[test_field_indices].ravel()
    X_train = X[train_field_indices]
    X_test = X[test_field_indices]

    return X_train, y_train, X_test, y_test

In [16]:
def cross_validate_lofo(df, drop_columns, train_model_func, reg=0.0, keras=True): 
    y_hat = np.array([])
    fields = df['Field_ID'].unique()
    
    for field in fields:
        X_train, y_train, X_test, y_test = train_test_split_lofo(df, drop_columns, field)
        X_train, X_test = normalize_X(X_train, X_test)
        model = train_model_func(X_train, y_train, X_test, y_test, reg=reg, verbose=0)
        y_hat = np.append(y_hat, model.predict(X_test))
    
    y = df['salinity']
    mse = mean_squared_error(y, y_hat)
    return y_hat, mse

In [17]:
def cross_validate_lofo_sklearn(df, drop_columns, model, normalize=False, **kwargs):
    y_hat = np.array([])
    fields = df['Field_ID'].unique()
    
    for field in fields:
        X_train, y_train, X_test, y_test = train_test_split_lofo(df, drop_columns, field)
        if(normalize): X_train, X_test = normalize_X(X_train, X_test)
        rf = model(**kwargs)
        rf.fit(X_train, y_train)
        y_hat = np.append(y_hat, rf.predict(X_test))
    
    y = df['salinity']
    mse = mean_squared_error(y, y_hat)
    return y_hat, mse

### Random Forest
#### 'Grid Search'

In [18]:
# Define X and y
y = df_small['salinity'].values
drop_columns = [0, 1]
X = df_small.drop(df_small.columns[drop_columns], axis=1).values

In [26]:
# model
model = RandomForestRegressor
parameter_grid = {'n_estimators': [100], 'min_samples_leaf': [1, 2, 10, 20, 50, 100, 200, 300, 400, 500],
                 'max_features': [2, 5, 7]}
kwarg_list = [dict(zip(parameter_grid, v)) for v in product(*parameter_grid.values())]

In [27]:
kwarg_list

[{'max_features': 2, 'min_samples_leaf': 1, 'n_estimators': 100},
 {'max_features': 5, 'min_samples_leaf': 1, 'n_estimators': 100},
 {'max_features': 7, 'min_samples_leaf': 1, 'n_estimators': 100},
 {'max_features': 2, 'min_samples_leaf': 2, 'n_estimators': 100},
 {'max_features': 5, 'min_samples_leaf': 2, 'n_estimators': 100},
 {'max_features': 7, 'min_samples_leaf': 2, 'n_estimators': 100},
 {'max_features': 2, 'min_samples_leaf': 10, 'n_estimators': 100},
 {'max_features': 5, 'min_samples_leaf': 10, 'n_estimators': 100},
 {'max_features': 7, 'min_samples_leaf': 10, 'n_estimators': 100},
 {'max_features': 2, 'min_samples_leaf': 20, 'n_estimators': 100},
 {'max_features': 5, 'min_samples_leaf': 20, 'n_estimators': 100},
 {'max_features': 7, 'min_samples_leaf': 20, 'n_estimators': 100},
 {'max_features': 2, 'min_samples_leaf': 50, 'n_estimators': 100},
 {'max_features': 5, 'min_samples_leaf': 50, 'n_estimators': 100},
 {'max_features': 7, 'min_samples_leaf': 50, 'n_estimators': 100},
 

In [28]:
%%time
mse_list = []
predicted_list = []

for index, kwargs in enumerate(kwarg_list):
    print("Working on:\t" + str(kwargs))
    y_hat, mse = cross_validate_lofo_sklearn(df_small, drop_columns, model, **kwargs) 
    predicted_list.append(y_hat)
    mse_list.append(mse)
    print("\t\tMSE: " + str(mse))

Working on:	{'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': 2}
		MSE: 29.42293423909455
Working on:	{'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': 5}
		MSE: 27.02047200843944
Working on:	{'n_estimators': 100, 'min_samples_leaf': 1, 'max_features': 7}
		MSE: 23.62590235933564
Working on:	{'n_estimators': 100, 'min_samples_leaf': 2, 'max_features': 2}
		MSE: 29.075711353690014
Working on:	{'n_estimators': 100, 'min_samples_leaf': 2, 'max_features': 5}
		MSE: 27.844239071129923
Working on:	{'n_estimators': 100, 'min_samples_leaf': 2, 'max_features': 7}
		MSE: 24.31982303756936
Working on:	{'n_estimators': 100, 'min_samples_leaf': 10, 'max_features': 2}
		MSE: 28.518446864992704
Working on:	{'n_estimators': 100, 'min_samples_leaf': 10, 'max_features': 5}
		MSE: 25.7803496359816
Working on:	{'n_estimators': 100, 'min_samples_leaf': 10, 'max_features': 7}
		MSE: 26.0416354410074
Working on:	{'n_estimators': 100, 'min_samples_leaf': 20, 'max_features': 2}
		MSE: 28.

In [29]:
best_run_index = np.argmin(mse_list)
params = kwarg_list[best_run_index]
y_hat = predicted_list[best_run_index]
mse = mse_list[best_run_index]

In [30]:
print('best run index:', best_run_index)
print('MAE: %.2f' % mean_absolute_error(y, y_hat))
print('MSE: %.2f' % mean_squared_error(y, y_hat))
print('R_squared: %.2f' % r2_score(y, y_hat))

best run index: 2
MAE: 3.33
MSE: 24.04
R_squared: 0.36


In [31]:
kwarg_list[best_run_index]

{'max_features': 7, 'min_samples_leaf': 1, 'n_estimators': 100}